In [1]:
# -*- coding: utf-8 -*-
import sys
import cv2
import mediapipe as mp
import numpy as np
from PIL import ImageFont, ImageDraw, Image
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import deque
import time

# PyQt5 관련 모듈
from PyQt5.QtWidgets import (
    QWidget, QLabel, QTextEdit, QHBoxLayout, QVBoxLayout, QPushButton, QDialog, QTextBrowser, QShortcut, QApplication, QPlainTextEdit, QLineEdit, QComboBox
)
from PyQt5.QtGui import QImage, QPixmap, QKeySequence
from PyQt5.QtCore import QThread, pyqtSignal, Qt, QPoint

# 키보드 제어 라이브러리 (사용되지 않으므로 주석 처리 가능)
# import pyautogui
# import pyperclip

from pathlib import Path

# ======================= PART 01. 수어 인식 모델 및 데이터 처리 로직 =======================

# >> 상수 및 설정 <<
consonant_labels = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
vowel_labels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ','ㅣ','ㅚ','ㅟ','ㅢ']
command_labels = ['shift', 'space', 'b_space', 'end']
labels = consonant_labels + vowel_labels + command_labels

font_path = "C:/Windows/Fonts/gulim.ttc"
dataset_file = 'combine_4.csv'

first_spelling = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
second_spelling = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
last_spelling = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

start_kr = 44032
end_kr = 55203

def is_hangul(char):
    return start_kr <= ord(char) <= end_kr

def decompose(char):
    if not is_hangul(char): return None, None, None
    code = ord(char) - start_kr; last_index = code % 28; code //= 28; second_index = code % 21; first_index = code // 21
    return first_spelling[first_index], second_spelling[second_index], last_spelling[last_index]

def compose(first, second, last=' '):
    try:
        first_index, second_index, last_index = first_spelling.index(first), second_spelling.index(second), last_spelling.index(last)
        code = start_kr + (first_index * 588) + (second_index * 28) + last_index
        return chr(code)
    except (ValueError, IndexError):
        return None

class HangulAssembler:
    def __init__(self):
        self.full_text = ""; self.command_shift = False
        self.double_consonant_map = {'ㄱ': 'ㄲ', 'ㄷ': 'ㄸ', 'ㅂ': 'ㅃ', 'ㅅ': 'ㅆ', 'ㅈ': 'ㅉ'}
        self.complex_last_map = {('ㄱ', 'ㅅ'): 'ㄳ', ('ㄴ', 'ㅈ'): 'ㄵ', ('ㄴ', 'ㅎ'): 'ㄶ', ('ㄹ', 'ㄱ'): 'ㄺ', ('ㄹ', 'ㅁ'): 'ㄻ', ('ㄹ', 'ㅂ'): 'ㄼ', ('ㄹ', 'ㅅ'): 'ㄽ', ('ㄹ', 'ㅌ'): 'ㄾ', ('ㄹ', 'ㅍ'): 'ㄿ', ('ㄹ', 'ㅎ'): 'ㅀ', ('ㅂ', 'ㅅ'): 'ㅄ'}
        self.dipthong_map = {('ㅗ', 'ㅏ'): 'ㅘ', ('ㅗ', 'ㅐ'): 'ㅙ', ('ㅗ', 'ㅣ'): 'ㅚ', ('ㅜ', 'ㅓ'): 'ㅝ', ('ㅜ', 'ㅔ'): 'ㅞ', ('ㅜ', 'ㅣ'): 'ㅟ', ('ㅡ', 'ㅣ'): 'ㅢ'}
        self.last_decompose_map = {v: k for k, v in self.complex_last_map.items()}

    def add_char(self, char):
        if char in command_labels: self._process_command(char)
        elif char in consonant_labels: self._process_consonant(char)
        elif char in vowel_labels: self._process_vowel(char)
        return self.full_text

    def _process_command(self, char):
        if char == 'shift': self.command_shift = True
        elif char == 'space': self.full_text += " "
        elif char == 'b_space': self._process_backspace()
        elif char == 'end': pass

    def get_current_text_and_reset(self):
        text_to_send = self.full_text
        self.full_text = ""
        self.command_shift = False
        return text_to_send

    def _process_backspace(self):
        if not self.full_text: return
        last_char = self.full_text[-1]
        if not is_hangul(last_char): self.full_text = self.full_text[:-1]; return
        first, second, last = decompose(last_char)
        if last != ' ':
            if last in self.last_decompose_map: new_char = compose(first,second, self.last_decompose_map[last][0])
            else: new_char = compose(first, second)
            self.full_text = self.full_text[:-1] + new_char
        elif second: self.full_text = self.full_text[:-1] + first
        else: self.full_text = self.full_text[:-1]

    def _process_consonant(self, char):
        if self.command_shift and char in self.double_consonant_map: char = self.double_consonant_map[char]
        self.command_shift = False
        last_char = self.full_text[-1] if self.full_text else None
        if last_char and is_hangul(last_char):
            first, second, last = decompose(last_char)
            if last == ' ' and char in last_spelling: self.full_text = self.full_text[:-1] + compose(first, second, char); return
            elif last in last_spelling and (last, char) in self.complex_last_map: self.full_text = self.full_text[:-1] + compose(first, second, self.complex_last_map[(last, char)]); return
        self.full_text += char

    def _process_vowel(self, char):
        self.command_shift = False
        last_char = self.full_text[-1] if self.full_text else None
        if not last_char: self.full_text += char; return
        if last_char in first_spelling: self.full_text = self.full_text[:-1] + compose(last_char, char); return
        if is_hangul(last_char):
            first, second, last = decompose(last_char)
            if last != ' ':
                if last in self.last_decompose_map:
                    last_1, last_2 = self.last_decompose_map[last]; syllable_1 = compose(first, second, last_1); syllable_2 = compose(last_2, char)
                    self.full_text = self.full_text[:-1] + syllable_1 + syllable_2
                else: syllable_1 = compose(first, second); syllable_2 = compose(last, char); self.full_text = self.full_text[:-1] + syllable_1 + syllable_2
                return
            else:
                if (second, char) in self.dipthong_map: self.full_text = self.full_text[:-1] + compose(first, self.dipthong_map[(second, char)]); return
        self.full_text += char

mp_hands = mp.solutions.hands; mp_drawing = mp.solutions.drawing_utils
def putText_korean(image, text, pos, font_path, font_size, color):
    img_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)); draw = ImageDraw.Draw(img_pil); font = ImageFont.truetype(font_path, font_size); draw.text(pos, text, font=font, fill=tuple(color[::-1]))
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
def calculate_angles(joint):
    v1=joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]; v2=joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]; v=v2-v1; v=v/np.linalg.norm(v,axis=1)[:,np.newaxis]
    return np.degrees(np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))).astype(np.float32)
def calculate_distances(joint):
    thumb_tip = joint[4]; other_tips = joint[[8, 12, 16, 20]]; distances = np.linalg.norm(other_tips - thumb_tip, axis=1)
    return distances.astype(np.float32)
def calculate_orientation_vectors(joint):
    v_direction=joint[9]-joint[0]; v_direction=np.zeros(3) if np.linalg.norm(v_direction)==0 else v_direction/np.linalg.norm(v_direction)
    v1,v2=joint[5]-joint[0],joint[17]-joint[0]; v_normal=np.cross(v1,v2); v_normal=np.zeros(3) if np.linalg.norm(v_normal)==0 else v_normal/np.linalg.norm(v_normal)
    return np.concatenate([v_direction, v_normal]).astype(np.float32)
def load_and_preprocess(dataset_file):
    print("데이터셋 로드 및 전처리 시작...")
    try:
        labels_str = np.genfromtxt(dataset_file, delimiter=',', skip_header=1, usecols=0, encoding="UTF-8", dtype=str)
        landmarks_data = np.genfromtxt(dataset_file, delimiter=',', skip_header=1, usecols=range(1, 127), encoding="UTF-8").astype(np.float32)
    except Exception as e:
        print(f"데이터 파일 로드 오류: {e}")
        return None, None, None

    all_features = []
    for row in landmarks_data:
        lh_landmarks = row[:63].reshape(21, 3); rh_landmarks = row[63:].reshape(21, 3)

        lh_angles = calculate_angles(lh_landmarks) if np.any(lh_landmarks) else np.zeros(15, dtype=np.float32)
        rh_angles = calculate_angles(rh_landmarks) if np.any(rh_landmarks) else np.zeros(15, dtype=np.float32)
        lh_coords = (lh_landmarks[1:] - lh_landmarks[0]).flatten() if np.any(lh_landmarks) else np.zeros(60, dtype=np.float32)
        rh_coords = (rh_landmarks[1:] - rh_landmarks[0]).flatten() if np.any(rh_landmarks) else np.zeros(60, dtype=np.float32)
        lh_distances = calculate_distances(lh_landmarks) if np.any(lh_landmarks) else np.zeros(4, dtype=np.float32)
        rh_distances = calculate_distances(rh_landmarks) if np.any(rh_landmarks) else np.zeros(4, dtype=np.float32)
        lh_orientation = calculate_orientation_vectors(lh_landmarks) if np.any(lh_landmarks) else np.zeros(6, dtype=np.float32)
        rh_orientation = calculate_orientation_vectors(rh_landmarks) if np.any(rh_landmarks) else np.zeros(6, dtype=np.float32)

        features = np.concatenate([lh_angles, rh_angles, lh_coords, rh_coords, lh_distances, rh_distances, lh_orientation, rh_orientation])
        all_features.append(features)

    all_features = np.array(all_features, dtype=np.float32)
    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels_str)

    print("데이터 전처리 완료!")
    return all_features, encoded_labels, encoder

def train_model(dataset_file):
    X, y, encoder = load_and_preprocess(dataset_file)
    if X is None: return None, None
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    # 1. 모델 학습
    model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    print("--- 랜덤 포레스트 모델 학습 시작 ---"); model.fit(X_train, y_train)

    # 2. 예측
    y_pred = model.predict(X_test)

    # 3. 정확도(Accuracy) 계산
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n--- 학습 완료 ---\n모델 테스트 정확도: {accuracy * 100:.2f}%")

    # 4. F1 score 계산
    f1 = f1_score(y_test, y_pred, average='macro')
    print("F1 score:", f1)
    return model, encoder

# ======================= PART 02. PyQt5 GUI 및 영상 처리 스레드 =======================
class VideoThread(QThread):
    change_pixmap_signal = pyqtSignal(np.ndarray)
    update_text_signal = pyqtSignal(str)

    def __init__(self, model, encoder):
        super().__init__()
        self._run_flag = True
        self.model = model
        self.encoder = encoder
        self.last_rec_time = 0
        self.rec_cool_time = 3.0
        self.display_duration = 3.0
        self.cap = cv2.VideoCapture(0)
        self._is_paused = False # <<< 추가: 일시정지 상태 플래그

    # <<< 추가: 일시정지/재시작을 위한 메서드 >>>
    def pause(self):
        self._is_paused = True

    def resume(self):
        self._is_paused = False

    def run(self):
        history = deque(maxlen=5)
        display_label = ""
        display_start_time = None
        
        with mp_hands.Hands(max_num_hands=2,
                              min_detection_confidence=0.5,
                              min_tracking_confidence=0.5) as hands:
            while self._run_flag and self.cap.isOpened():
                # <<< 추가: 일시정지 상태이면 루프를 건너뜀 >>>
                if self._is_paused:
                    time.sleep(0.01) # CPU 사용 방지를 위해 잠시 대기
                    continue

                ret, frame = self.cap.read()
                if not ret: break
                
                frame = cv2.flip(frame, 1)
                result = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                
                current_time = time.time()
                guide_text = "손을 보여주세요"
                hands_present = False

                if result.multi_hand_landmarks:
                    hands_present = True
                    # (이하 특징 추출 로직은 동일)
                    init_zeros = {'angles': np.zeros(15), 'coords': np.zeros(60), 'distances': np.zeros(4), 'orientation': np.zeros(6)}
                    lh_features, rh_features = init_zeros.copy(), init_zeros.copy()
                    for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
                        handedness = result.multi_handedness[i].classification[0].label
                        joint = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
                        features = {'angles': calculate_angles(joint),'coords': (joint[1:]-joint[0]).flatten(),'distances': calculate_distances(joint),'orientation': calculate_orientation_vectors(joint)}
                        if handedness == "Left": lh_features = features
                        elif handedness == "Right": rh_features = features
                        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    feature_vector = np.concatenate([lh_features['angles'], rh_features['angles'],lh_features['coords'], rh_features['coords'],lh_features['distances'], rh_features['distances'],lh_features['orientation'], rh_features['orientation']]).reshape(1, -1)
                    predicted_label = self.encoder.inverse_transform(self.model.predict(feature_vector))[0]
                    history.append(predicted_label)
                    
                    if len(history) == 5 and len(set(history)) == 1:
                        if current_time - self.last_rec_time > self.rec_cool_time:
                            mapped_label = history[-1]
                            self.update_text_signal.emit(mapped_label)
                            display_label, display_start_time, self.last_rec_time = mapped_label, current_time, current_time
                            history.clear()
                
                if hands_present:
                    if display_start_time and ((current_time - display_start_time) < self.display_duration):
                        display_text = display_label
                    else: display_text = "인식 중..."
                else: display_text = guide_text
                
                frame = putText_korean(frame, display_text, (50, 420), font_path, 40, (0, 255, 0))
                self.change_pixmap_signal.emit(frame)
        
        self.cap.release()

    def stop(self):
        self._run_flag = False
        self.wait()

class HelpWindow(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("도움말")
        self.resize(600, 650)
        lay, title, body = QVBoxLayout(self), QLabel("<b>수어 예시</b>"), QTextBrowser()
        img_path = Path(r"C:\Users\KCCISTC\Desktop\workspace\project_1\module_hand\hand_img.png")
        if not img_path.exists(): body.setText("도움말 이미지 파일을 찾을 수 없습니다.")
        else: body.setHtml(f'<h3>수어 예시표 입니다.</h3><br><img src="{img_path.as_uri()}" width="550"><br>')
        lay.addWidget(title); lay.addWidget(body)

class SettingsWindow(QDialog):
    speed_changed = pyqtSignal(float)
    def __init__(self, current_speed, parent=None):
        super().__init__(parent)
        self.setWindowTitle("설정"); self.resize(350, 200)
        self.info_label1 = QLabel("인식 속도 조절"); self.info_label2 = QLabel("숫자가 낮을수록 인식 간격이 짧아져 빨라집니다.")
        self.speed_combo = QComboBox(); self.apply_button = QPushButton("적용")
        self.speed_options = {"매우 느림 (5.0초)": 5.0, "느림 (4.0초)": 4.0, "보통 (3.0초)": 3.0, "빠름 (2.0초)": 2.0, "매우 빠름 (1.0초)": 1.0}
        for text, value in self.speed_options.items(): self.speed_combo.addItem(text, value)
        for i, value in enumerate(self.speed_options.values()):
            if value == current_speed: self.speed_combo.setCurrentIndex(i); break
        layout = QVBoxLayout(self); layout.addWidget(self.info_label1); layout.addWidget(self.info_label2)
        layout.addWidget(self.speed_combo); layout.addStretch(1); layout.addWidget(self.apply_button)
        self.apply_button.clicked.connect(self.apply_settings)
    def apply_settings(self):
        selected_speed = self.speed_combo.currentData(); self.speed_changed.emit(selected_speed); self.accept()

class SignLanguageTranslatorApp(QWidget):
    def __init__(self, model, encoder):
        super().__init__()
        self.setWindowTitle("수어 번역 프로그램")
        self.current_rec_speed = 3.0 
        self.is_paused = False # <<< 추가: 일시정지 상태 플래그

        # --- UI 위젯 생성 ---
        self.camera_view = QLabel(self); self.camera_view.setObjectName("cameraView"); self.camera_view.setMinimumSize(600, 480)
        self.pause_button = QPushButton("일시정지") # <<< 추가: 일시정지 버튼 생성
        self.settings_button = QPushButton("설정")
        self.help_button = QPushButton("도움말")
        self.log_box = QPlainTextEdit(self); self.log_box.setReadOnly(True)
        self.bottom_input = QLineEdit(self)

        style_sheet = """
            QWidget { background-color: #2E2E2E; color: #F0F0F0; font-family: "Malgun Gothic"; font-size: 11pt; }
            #cameraView { border: 2px solid #00A0A0; border-radius: 5px; }
            QPushButton { background-color: #555555; border: 1px solid #777777; padding: 5px; border-radius: 5px; }
            QPushButton:hover { background-color: #6A6A6A; }
            QPushButton:pressed { background-color: #4A4A4A; }
            QPlainTextEdit { background-color: #3C3C3C; border: 1px solid #555555; border-radius: 5px; font-size: 12pt; padding: 5px; }
            QLineEdit { background-color: #3C3C3C; border: 1px solid #555555; border-radius: 5px; font-size: 12pt; padding: 5px; selection-background-color: #00A0A0; selection-color: white; }
            QComboBox { border: 1px solid #777777; border-radius: 3px; padding: 1px 18px 1px 3px; min-width: 6em; background-color: #555555;}
            QComboBox::drop-down { subcontrol-origin: padding; subcontrol-position: top right; width: 20px; border-left-width: 1px; border-left-color: #777777; border-left-style: solid; border-top-right-radius: 3px; border-bottom-right-radius: 3px; }
        """
        self.setStyleSheet(style_sheet)
        
        # --- 레이아웃 설정 ---
        button_layout = QHBoxLayout(); button_layout.addStretch(1)
        button_layout.addWidget(self.pause_button) # <<< 추가: 레이아웃에 일시정지 버튼 추가
        button_layout.addWidget(self.settings_button); button_layout.addWidget(self.help_button)
        right_pane_layout = QVBoxLayout(); right_pane_layout.addLayout(button_layout); right_pane_layout.addWidget(self.log_box, stretch=10); right_pane_layout.addWidget(self.bottom_input, stretch=1)
        right_widget = QWidget(); right_widget.setLayout(right_pane_layout); right_widget.setFixedWidth(350)
        main_layout = QHBoxLayout(self); main_layout.addWidget(self.camera_view, stretch=2); main_layout.addWidget(right_widget, stretch=1)
        self.setLayout(main_layout); self.resize(980, 520)

        # --- 시그널과 슬롯 연결 ---
        self.pause_button.clicked.connect(self.toggle_pause_resume) # <<< 추가: 일시정지 버튼 클릭 시그널 연결
        self.help_button.clicked.connect(self.toggle_help_window)
        self.settings_button.clicked.connect(self.open_settings_window)
        self.bottom_input.returnPressed.connect(self.finalize_sentence)

        # --- 기타 초기화 ---
        self.help_window, self.settings_window = None, None
        self.assembler = HangulAssembler()
        self.thread = VideoThread(model, encoder)
        self.thread.change_pixmap_signal.connect(self.update_image)
        self.thread.update_text_signal.connect(self.update_text)
        self.thread.start()

        # 단축키
        self.quit_shortcut = QShortcut(QKeySequence('q'), self); self.quit_shortcut.activated.connect(self.close)
        self.help_shortcut = QShortcut(QKeySequence(Qt.Key_F1), self); self.help_shortcut.activated.connect(self.toggle_help_window)

        # 좌상단 고정
        self.move(80, 80)
    
    # <<< 추가: 일시정지/재시작 토글 메서드 >>>
    def toggle_pause_resume(self):
        self.is_paused = not self.is_paused
        if self.is_paused:
            self.thread.pause()
            self.pause_button.setText("다시 시작")
        else:
            self.thread.resume()
            self.pause_button.setText("일시정지")

    def open_settings_window(self):
        if self.settings_window is None or not self.settings_window.isVisible():
            self.settings_window = SettingsWindow(self.current_rec_speed, self)
            self.settings_window.speed_changed.connect(self.update_recognition_speed)
            self.settings_window.show()
        else:
            self.settings_window.activateWindow()

    def update_recognition_speed(self, new_speed):
        self.current_rec_speed = new_speed; self.thread.rec_cool_time = new_speed; self.thread.display_duration = new_speed
        print(f"인식 속도가 {new_speed}초로 변경되었습니다.")

    def finalize_sentence(self):
        text = self.bottom_input.text()
        if text: self.log_box.appendPlainText(text)
        self.bottom_input.clear(); self.assembler = HangulAssembler()
        
    def update_text(self, new_char):
        if new_char == 'end':
            text_to_send = self.assembler.get_current_text_and_reset()
            if text_to_send:
                self.log_box.appendPlainText(text_to_send)
            self.bottom_input.clear()
            return
        current_text = self.assembler.add_char(new_char)
        self.bottom_input.setText(current_text)
        
    def toggle_help_window(self):
        if self.help_window is None or not self.help_window.isVisible():
            self.help_window = HelpWindow(self); self.help_window.setModal(False)
            self.help_window.move(self.frameGeometry().topRight() + QPoint(10, 0)); self.help_window.show()
        else:
            self.help_window.close()
            
    def update_image(self, cv_img):
        qt_img = self.convert_cv_qt(cv_img); self.camera_view.setPixmap(qt_img)
        
    def convert_cv_qt(self, cv_img):
        rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB); h, w, ch = rgb_image.shape; p = QPixmap.fromImage(QImage(rgb_image.data, w, h, ch * w, QImage.Format_RGB888))
        return p.scaled(self.camera_view.width(), self.camera_view.height(), Qt.KeepAspectRatio)
        
    def closeEvent(self, event):
        if self.help_window and self.help_window.isVisible(): self.help_window.close()
        if self.settings_window and self.settings_window.isVisible(): self.settings_window.close()
        self.thread.stop(); event.accept(); cv2.destroyAllWindows(); cv2.waitKey(1)

# ======================= PART 03. 메인 실행 부분 =======================
if __name__ == "__main__":
    trained_model, label_encoder = train_model(dataset_file)
    if trained_model and label_encoder:
        app = QApplication(sys.argv)
        window = SignLanguageTranslatorApp(trained_model, label_encoder)
        window.show()
        sys.exit(app.exec_())
    else:
        print("모델 학습에 실패하여 프로그램을 종료합니다.")

데이터셋 로드 및 전처리 시작...
데이터 전처리 완료!
--- 랜덤 포레스트 모델 학습 시작 ---

--- 학습 완료 ---
모델 테스트 정확도: 99.45%


SystemExit: 0

C:\ProgramData\anaconda3\envs\project\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
